# EJP Common template - Data Query Module
Upload the database and then use the drop-down to select the information you want to retrieve. A link to a temporary excel/csv file will be provided for you to download.

Three types of exports:
- from **SQL request**: write down your SQL request to be executed on the database (most flexible)
- as **single table**: select the tabs and measurements you want to be included then everything will be available in one single excel sheet with stacked data (most simple)
- as **control vs treatment**: select tabs you want included then check which 'Ctrl vs treatment' you want to observe. (Identification of the paired will be done automatically from a matrix comparison of treatment caracteristic for the tab selected -> may be more complex for the 'crops' tab actually)

In [66]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import matplotlib.dates as mdates
from xlsx2csv import Xlsx2csv
import tempfile
import os
import base64
    
from ipywidgets import (FileUpload, Button, Output, Dropdown, RadioButtons,
                        SelectMultiple, VBox, HBox, Layout, Checkbox, Label, Text,
                       FloatText, FloatRangeSlider)
from IPython.display import FileLink, HTML

sheetNames = ['experiment', 'reference', 'treatment', 'soil', 'tillage', 'crops',
             'fertilization', 'amendment', 'irrigation', 'pest-weed', 'grazing',
             'soil-crop-measurement', 'data', 'dropDownList']

dtypes = {
    'Experiment ID': 'string',
    'Treatment ID': 'string',
    'Reference treatment': 'string',
         }

def camelCase(s):
    s = s.split()
    return s[0].lower() + ''.join(i.capitalize() for i in s[1:])
def underCase(s):
    s = s.split()
    return '_'.join(i.lower() for i in s)

In [12]:
dfdic = readExcel('../../data/carboseq-wp2-db.xlsx')
dfdic.keys()

Reading in Excel file...done (0.53s)


dict_keys(['README', 'description', 'experiment', 'reference', 'soil-type', 'treatment', 'tillage', 'crops', 'amendment', 'irrigation', 'data-crop', 'data-soil', 'dropDownList'])

In [19]:
dbdic = {
    'experiment': {
        'Latitude': 'number',
        'Longitude': 'number',
        'Country': 'choice',
        'Land use prior experiment': 'choice'
    },
    'reference': {
        'Publication type': 'choice',
        'Publication first author': 'string',
        'Publication year': 'number',
        'Publication title': 'string',
        'Publication journal': 'string'
    },
    'soil-type': {
        'Top depth of layer': 'number',
        'Bottom depth of layer': 'number',
        'Clay (< 0.002 mm)': 'number',
        'Silt (0.002 - 0.05 mm)': 'number',
        'Sand (0.05 - 2 mm)': 'number',
        'Gravel (> 2 mm)': 'number',
        'Soil texture USDA': 'choice',
        'Soil group WRB': 'choice',
        'Soil group WRB qualifier': 'choice',
        'Soil group WRB specifier': 'choice',
        'Soil type USDA': 'choice',
        'Soil type USDA qualifier': 'choice',
    },
    'treatment': {
        'Land use': 'choice',
        'Year started': 'number',
        'Year ended': 'number',
        'Crop rotation': 'choice'
    },
    'tillage': {
        'Tillage system': 'choice',
        'Tillage method': 'choice',
        'Tillage depth': 'number',
        'Permanent soil area covered by residues or crops': 'choice',
        'Tillage period': 'choice',
    },
    'crops': {
        'Crop type': 'choice',
        'Cropping system': 'choice',
        'Crop': 'choice',
        'Harvesting/Termination method': 'choice',
        'Harvesting frequency': 'number',
        'Sowing period': 'choice',
        'Harvesting/Termination period': 'choice',
        'Residues removal': 'choice',
        'Residues incorporation': 'choice', 
        'Residues burning': 'choice'
    },
    'amendment': {
        'Type of fertilizer/amendment': 'choice',
        'Fertilizer/Amendment application rate': 'number',
        'Fertilizer/Amendment application rate units': 'choice',
        'Fertilizer/Amendment application method': 'choice',
        'Amendment water content': 'number',
        'Amendment C': 'number',
        'Amendment N': 'number',
        'Amendment P': 'number',
        'Amendment K': 'number',
    },
    'irrigation': {
        'Irrigation method': 'choice',
        'Amount of water': 'number',
        'Irrigation frequency': 'number',
        'Irrigation water': 'number',
        'Drainage system': 'choice',
        'Drainage spacing': 'number',
        'Drainage depth': 'number',
    },
    'data-crop': {
        'Sampling year': 'number',
        'Harvested yield': 'number',
        'Harvested yield water content': 'number',
        'Harvested yield water content amount': 'number',
        'Residue above-ground': 'number',
        'Residue stubble': 'number',
        'Residue roots': 'number',
        'Residue sampling method': 'string',
        'Below-ground sampling depth': 'number',
    },
    'data-soil': {
        'Sampling year': 'number',
        'Depth from': 'number',
        'Depth to': 'number',
        'Time-serie available': 'choice',
        'SOC conc': 'number',
        'SOC conc SD': 'number',
        'SOC conc SE': 'number',
        'SOC conc nb samples': 'number',
        'Analysis method': 'choice',
        'Bulk density': 'number',
        'Bulk density method': 'choice',
        'Bulk density SD': 'number',
        'Bulk density SE': 'number',
        'Bulk density nb samples': 'number',
        'SOC stock': 'number',
        'SOC stock SD': 'number',
        'SOC stock SE': 'number',
        'SOC stock nb samples': 'number',
        'pH': 'number',
        'pH method': 'choice'
    }
}

In [2]:
def readExcel2(data):
    t0 = time.time()
    print('Reading in Excel file...', end='')
    a = Xlsx2csv(data, outputencoding="utf-8")
    dfdic = {}
    with tempfile.TemporaryDirectory() as td:
        a.convert(td, sheetid=0)
        for i, sheet in enumerate(sheetNames):
            fname = os.path.join(td, sheet + '.csv')
            df = pd.read_csv(fname, skiprows=[0,1,3], dtype=dtypes)
            dfdic[sheet] = df.dropna(how='all')
        dfdic['data'] = dfdic['data'].dropna(axis=1)
    datetimeList = [('crops', 'Sowing date'),
                    ('crops', 'Harvesting date'),
                    ('tillage', 'Tillage date'),
                    ('amendment', 'Amendment date'),
                    ('fertilization', 'Fertilizer application date'),
                    ('irrigation', 'Irrigation date'),
                    ('pest', 'Pesticide application date'),
                    ('measurement', 'Sampling date'),
                    ('data', 'Date')
                   ]
    for row in datetimeList:
        df = dfdic[row[0]]
        if row[0] in df.columns:
            df[row[1]] = pd.to_datetime(df[row[1]])
    dfdic['data'] = dfdic['data'].dropna(axis=1)
    print('done ({:.2f}s)'.format(time.time() - t0))
    return dfdic
#dfdic = readExcel2('../../ejp-common-template2.xlsx')

In [4]:
def readExcel(fname):
    t0 = time.time()
    print('Reading in Excel file...', end='')
    if fname[:4] == 'http': # it's a google sheet url
        fname = '/'.join(fname.split('/')[:-1] + ['export?format=xlsx'])
    dfdic = pd.read_excel(fname, sheet_name=None, skiprows=[0,1,3])
    print('done ({:.2f}s)'.format(time.time() - t0))
    return dfdic

#dfdic = readExcel('../../../ejp-wp7/ejp-common-template2.xlsx')

In [64]:
dfdic = readExcel('../../data/carboseq-wp2-db.xlsx')

Reading in Excel file...done (0.32s)


In [103]:
# filter experiments
class RowFilter(object):
    def __init__(self, filters, dfdic):
        self.sheetDropdown = Dropdown(options=list(dbdic.keys()), layout=Layout(width='15%'))
        self.sheetDropdown.observe(self.sheetDropdownFunc, names='value')
        self.colDropdown = Dropdown(options=list(dbdic['experiment'].keys()), layout=Layout(width='15%'))
        self.colDropdown.observe(self.colDropdownFunc, names='value')
        self.rmBtn = Button(description='Remove')
        self.rmBtn.on_click(self.rmBtnFunc)
        self.hbox = HBox([self.sheetDropdown, self.colDropdown, self.rmBtn], layout=Layout(display='flex'))
        self.filters = filters
        self.index = len(self.filters.children) - 1
        self.filters.children = self.filters.children[:-1] + (self.hbox, self.filters.children[-1])
        self.dfdic = dfdic
        self.opts = None
        self.buildOptions('experiment', 'Latitude')

    def sheetDropdownFunc(self, a):
        self.colDropdown.options = list(dbdic[a['new']].keys())

    def colDropdownFunc(self, a):
        sheet = self.sheetDropdown.value
        col = a['new']
        self.buildOptions(sheet, col)

    def buildOptions(self, sheet, col):
        typ = dbdic[sheet][col]
        print('type', typ)
        if typ == 'number':
            vmin = dfdic[sheet][col].min()
            vmax = dfdic[sheet][col].max()
            r = vmax - vmin
            #opts = FloatRangeSlider(value=[vmin, vmax], vmin=vmin - 0.1*r, vmax=vmax + 0.1*r, step=0.02*r)
            self.opts = HBox([Label('min:'), FloatText(value=vmin, layout=Layout(width='30%')),
                              Label('max:'), FloatText(value=vmax, layout=Layout(width='30%'))])
        if typ == 'choice' or typ == 'string':
            choices = dfdic[sheet][col].dropna().unique()
            self.opts = VBox([Checkbox(value=True, description=a, indent=False)
                              for a in choices], layout=Layout(width='40%'))
        if len(self.hbox.children) > 3:
            self.hbox.children = self.hbox.children[:-2] + (self.opts, self.hbox.children[-1])
        else:
            self.hbox.children = self.hbox.children[:-1] + (self.opts, self.hbox.children[-1])

    def rmBtnFunc(self, a):
        b = list(self.filters.children)
        del b[self.index]
        self.filters.children = b

def addFilterBtnFunc(b):
    rowFilter = RowFilter(filters, dfdic)
addFilterBtn = Button(description='Add filter')
addFilterBtn.on_click(addFilterBtnFunc)

def filterDataBtnFunc(a):
    expids = pd.Series(dfdic['experiment']['Experiment ID'].unique())
    for child in filters.children[:-1]:
        sheet = child.children[0].value
        col = child.children[1].value
        typ = dbdic[sheet][col]
        if typ == 'number':
            print(child.children)
            vmin = child.children[2].children[1].value
            vmax = child.children[2].children[3].value
            ie = (dfdic[sheet][col] >= vmin) & (dfdic[sheet][col] <= vmax)
        else:
            choices = [b.description for b in child.children[2].children if b.value is True]
            ie = dfdic[sheet][col].isin(choices)
        expids = expids[expids.isin(dfdic[sheet][ie]['Experiment ID'].unique())]
    print('filtered expids:', expids)
filterDataBtn = Button(description='Filter data')
filterDataBtn.on_click(filterDataBtnFunc)

filters = VBox([HBox([addFilterBtn, filterDataBtn])])
display(filters)

In [20]:
# proof of concept of web-based input file check (on binder for instance)
dfdic = pd.DataFrame()
df = pd.DataFrame()

# upload button
def uploadBtnFunc(btn):
    global dfdic
    out.clear_output()
    with out:
        if gsurl.value != '':
            dfdic = readExcel(gsurl.value)
        elif len(uploadBtn.data) > 0:
            dfdic = readExcel(uploadBtn.data[0])
            #with open('t.xlsx', 'wb') as f: # faster but less robust
            #    f.write(upload.data[0])
            #dfdic = readExcel2('t.xlsx')
        if 'data' in dfdic.keys():
            measurements = dfdic['data']['Measurement'].unique()
            measurements = sorted(measurements)
            measChecks.children = [measChecks.children[0]] + \
                [Checkbox(value=False, indent=False, description=a) for a in measurements]
uploadBtn = FileUpload(accept='.xlsx', multiple=False)
#uploadBtn.observe(uploadBtnFunc, names='_counter')

# where to put the url of the Google Sheet
gsurl = Text()
#gsurl.observed(uploadBtnFunc, names='_')

loadBtn = Button(description='Load File/URL', style= {'button_color':'orange'})
loadBtn.on_click(uploadBtnFunc)

# output for displaying processing
out = Output()



# interactive visualization




# case of headers
headerRadio = RadioButtons(
    options=['Default (with space)','camelCase','under_case'],
    description="Headers:")

# list of sheets to include in the extracted table
sheetChecks = VBox([Checkbox(description='Select all', indent=False)] + 
                             [Checkbox(description=sheet, indent=False) for sheet in sheetNames[1:-3]])
def sheetChecksFunc(a):
    children = sheetChecks.children
    for child in children[1:]:
        child.value = children[0].value
sheetChecks.children[0].observe(sheetChecksFunc, names='value')

# list of variable to include in the extracted table
measChecks = VBox([
    Checkbox(description='Select all', indent=False)
])
def measChecksFunc(a):
    children = measChecks.children
    for child in children[1:]:
        child.value = children[0].value
measChecks.children[0].observe(measChecksFunc, names='value')

# display extracted links here
extracted = Output()




# choice between stacked or 'vs Control' layout for extracted table
treatmentRadio = RadioButtons(
    options=['Stacked','vs Control'],
    description='Treatments:')


# extract button (perform database extraction and merging of tables)
def extractBtnFunc(a):
    global dfdic, df
    stacked = treatmentRadio.value == 'Stacked'
    sheetCheck = [a.description for a in sheetChecks.children[1:] if a.value is True]
    measCheck = [a.description for a in measChecks.children[1:] if a.value is True]
    df = dfdic['experiment'] # always include 'experiment' tab
    for sheet in sheetCheck:
        # by default merge is performed on columns of same name (so perfect for our case)
        df = pd.merge(df, dfdic[sheet], how='outer')
    # add data to this
    if 'data' in dfdic.keys():
        dfdata = dfdic['data']
        ie = np.in1d(dfdata['Measurement'], measCheck)
        df = pd.merge(df, dfdata[ie], how='outer')
    
    if stacked is False:
        iref = df['Reference treatment'].isna()
        dft = df[~iref].copy() # treatment
        dfc = df[iref].copy() # control
        cols = np.array(['Rotation', 'Crops ID']) # experiment id included later
        ie = np.in1d(cols, df.columns)
        on = cols[ie].tolist() + ['Reference treatment']
        on = on + dfdic['experiment'].columns.tolist()
        dfc = dfc.drop('Reference treatment', axis=1)
        dfc = dfc.rename(columns={'Treatment ID': 'Reference treatment'})
        if 'Measurement' not in on:
            on += ['Measurement']
        df = pd.merge(dft, dfc, on=on, how='outer', suffixes=('_T', '_C'))
    if headerRadio.value == 'camelCase':
        df = df.rename(columns=dict(zip(df.columns, list(map(camelCase, df.columns)))))
    elif headerRadio.value == 'under_case':
        df = df.rename(columns=dict(zip(df.columns, list(map(underCase, df.columns)))))
    extracted.clear_output()
    with extracted:
        fname = 'extracted.xlsx'
        df.to_excel(fname, index=False)
        with open(fname,  'rb') as f:
            data = f.read()
        b64 = base64.b64encode(data)
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload, title=fname, filename=fname)
        display(HTML(html))
extractBtn = Button(description = 'Extract data')
extractBtn.on_click(extractBtnFunc)


# layout
optionLayout = HBox([
    VBox([Label('Options:'), treatmentRadio, headerRadio]),
    VBox([Label('Sheets:'), sheetChecks]),
    VBox([Label('Measurements:'), measChecks], layout=Layout(width='40%')),
])
mainLayout = VBox([
    HBox([uploadBtn, Label('OR Google Sheet URL:'), gsurl]),
    loadBtn,
    out,
    optionLayout,
    extractBtn,
    extracted
])


display(mainLayout)

In [20]:
# proof of concept of web-based input file check (on binder for instance)
dfdic = pd.DataFrame()
df = pd.DataFrame()

# upload button
def uploadBtnFunc(btn):
    global dfdic
    out.clear_output()
    with out:
        if gsurl.value != '':
            dfdic = readExcel(gsurl.value)
        elif len(uploadBtn.data) > 0:
            dfdic = readExcel(uploadBtn.data[0])
            #with open('t.xlsx', 'wb') as f: # faster but less robust
            #    f.write(upload.data[0])
            #dfdic = readExcel2('t.xlsx')
        if 'data' in dfdic.keys():
            measurements = dfdic['data']['Measurement'].unique()
            measurement = sorted(measurements)
            measChecks.children = [measChecks.children[0]] + \
                [Checkbox(value=False, indent=False, description=a) for a in measurements]
uploadBtn = FileUpload(accept='.xlsx', multiple=False)
#uploadBtn.observe(uploadBtnFunc, names='_counter')

# where to put the url of the Google Sheet
gsurl = Text()
#gsurl.observed(uploadBtnFunc, names='_')

loadBtn = Button(description='Load File/URL', style= {'button_color':'orange'})
loadBtn.on_click(uploadBtnFunc)

# output for displaying processing
out = Output()

# choice between stacked or 'vs Control' layout for extracted table
treatmentRadio = RadioButtons(
    options=['Stacked','vs Control'],
    description='Treatments:')

# case of headers
headerRadio = RadioButtons(
    options=['Default (with space)','camelCase','under_case'],
    description="Headers:")

# list of sheets to include in the extracted table
sheetChecks = VBox([Checkbox(description='Select all', indent=False)] + 
                             [Checkbox(description=sheet, indent=False) for sheet in sheetNames[1:-3]])
def sheetChecksFunc(a):
    children = sheetChecks.children
    for child in children[1:]:
        child.value = children[0].value
sheetChecks.children[0].observe(sheetChecksFunc, names='value')

# list of variable to include in the extracted table
measChecks = VBox([
    Checkbox(description='Select all', indent=False)
])
def measChecksFunc(a):
    children = measChecks.children
    for child in children[1:]:
        child.value = children[0].value
measChecks.children[0].observe(measChecksFunc, names='value')

# display extracted links here
extracted = Output()

# extract button (perform database extraction and merging of tables)
def extractBtnFunc(a):
    global dfdic, df
    stacked = treatmentRadio.value == 'Stacked'
    sheetCheck = [a.description for a in sheetChecks.children[1:] if a.value is True]
    measCheck = [a.description for a in measChecks.children[1:] if a.value is True]
    df = dfdic['experiment'] # always include 'experiment' tab
    for sheet in sheetCheck:
        # by default merge is performed on columns of same name (so perfect for our case)
        df = pd.merge(df, dfdic[sheet], how='outer')
    # add data to this
    if 'data' in dfdic.keys():
        dfdata = dfdic['data']
        ie = np.in1d(dfdata['Measurement'], measCheck)
        df = pd.merge(df, dfdata[ie], how='outer')
    
    if stacked is False:
        iref = df['Reference treatment'].isna()
        dft = df[~iref].copy() # treatment
        dfc = df[iref].copy() # control
        cols = np.array(['Rotation', 'Crops ID']) # experiment id included later
        ie = np.in1d(cols, df.columns)
        on = cols[ie].tolist() + ['Reference treatment']
        on = on + dfdic['experiment'].columns.tolist()
        dfc = dfc.drop('Reference treatment', axis=1)
        dfc = dfc.rename(columns={'Treatment ID': 'Reference treatment'})
        if 'Measurement' not in on:
            on += ['Measurement']
        df = pd.merge(dft, dfc, on=on, how='outer', suffixes=('_T', '_C'))
    if headerRadio.value == 'camelCase':
        df = df.rename(columns=dict(zip(df.columns, list(map(camelCase, df.columns)))))
    elif headerRadio.value == 'under_case':
        df = df.rename(columns=dict(zip(df.columns, list(map(underCase, df.columns)))))
    extracted.clear_output()
    with extracted:
        fname = 'extracted.csv'
        df.to_csv(fname, index=False)
        with open(fname,  'rb') as f:
            data = f.read()
        b64 = base64.b64encode(data)
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload, title=fname, filename=fname)
        display(HTML(html))
        #display(FileLink('extracted.csv'))
extractBtn = Button(description = 'Extract data')
extractBtn.on_click(extractBtnFunc)


# layout
optionLayout = HBox([
    VBox([Label('Options:'), treatmentRadio, headerRadio]),
    VBox([Label('Sheets:'), sheetChecks]),
    VBox([Label('Measurements:'), measChecks], layout=Layout(width='40%')),
])
mainLayout = VBox([
    HBox([uploadBtn, Label('OR Google Sheet URL:'), gsurl]),
    loadBtn,
    out,
    optionLayout,
    extractBtn,
    extracted
])


display(mainLayout)